In [3]:
import pandas as pd
import os
import librosa
import numpy as np
import scipy
import re


#Pickling
from six.moves import cPickle as pickle

#HTTP
import requests
import json

#
import collections

import matplotlib.pyplot as plt
import matplotlib.style as ms
import librosa.display
import IPython.display as ipd
import seaborn
ms.use('seaborn-muted')
%matplotlib inline


In [4]:
def get_pattern(text):
    text = json.dumps(text)
    url = 'http://192.168.2.101:7878/api/get_patt'
    data = dict(input_tweets = text)
    resp = requests.post(url=url, data=data)
    r = json.loads(resp.text)
    return map(lambda x: x['pattern'],r)
    



def clean_text(text):
    punct_str = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~«»“…‘”'
    for p in punct_str:
        text = text.replace(p,' ')
    text = re.sub(' +', ' ', text)
    return text.lower().strip()
    

In [5]:
# full = generate_IEMOCAP_df()
data = pd.read_csv('data/IEMOCAP_sentences.csv',index_col=0)
# data.head(3)
emotional_mapping = {'ang': 0, 'sad': 1, 'exc': 2, 'neu': 3,'fru': 4,'hap': 5,'fea': 6,'sur': 7,'dis': 8, 'xxx':9,'oth':10}
data['emotion_code'] = data['emotion'].map( emotional_mapping ).astype(int)
data = data[data.emotion_code < 6]

In [10]:
data['text'] = data['text'].apply(clean_text)
# patterns = {}
# i = 0
# for sentence in sentences:
#     print(sentence)
# #     patterns[i] = get_pattern
#     clean_s = clean_text(sentence)
#     print(clean_s)
#     i = i+1
#     if(i > 5):
#         break
data.head(5)

,index,start_time,end_time,text,wav_path,alignment_path,emotion,valence,arousal,dominance,gender,emotion_code
0,Ses01F_impro01_F000,6.2901,8.2357,excuse me,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,neu,2.5,2.5,2.5,F,3
1,Ses01F_impro01_F001,10.0100,11.3925,yeah,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,neu,2.5,2.5,2.5,F,3
2,Ses01F_impro01_F002,14.8872,18.0175,is there a problem,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,neu,2.5,2.5,2.5,F,3
5,Ses01F_impro01_F005,27.4600,31.4900,well what's the problem let me change it,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,neu,2.5,3.5,2.0,F,3
6,Ses01F_impro01_F006,38.9650,43.5900,what i'm getting an id this is why i'm here my...,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,fru,2.0,3.5,3.5,F,4


In [14]:
patterns = {}
for index, row in data.iterrows():
    patterns[index] = get_pattern(list(row['text']))
    if (index > 10):
        break
        

In [19]:
list(data['text'][0]

['e', 'x', 'c', 'u', 's', 'e', ' ', 'm', 'e']

In [25]:
ang_sentences = data[data.emotion_code == 0]['text']
sad_sentences = data[data.emotion_code == 1]['text']
exc_sentences = data[data.emotion_code == 2]['text']
neu_sentences = data[data.emotion_code == 3]['text']
fru_sentences = data[data.emotion_code == 4]['text']
hap_sentences = data[data.emotion_code == 5]['text']

In [ ]:
counter = collections.Counter(list(sentences))
print('Overall')
print(counter.most_common(5))
counter = collections.Counter(list(ang_sentences))
print('Anger')
print(counter.most_common(5))
counter = collections.Counter(list(hap_sentences))
print('Happiness')
print(counter.most_common(5))

In [2]:
counter = collections.Counter(list(hap_sentences))
counter.most_common(5)

NameError: name 'collections' is not defined

In [35]:
counter = collections.Counter(list(hap_sentences))
counter.most_common()

[('Or not.', 11),
 ('[LAUGHTER]', 9),
 ('Every day since.', 7),
 ("What'll I do with a fortune?", 4),
 ('Yeah.', 4),
 ('And very much sillier.', 3),
 ('I guess this is why I came.', 3),
 ('What would I do with a fortune?', 3),
 ('yeah.', 3),
 ('Which particular one?', 3),
 ('[BREATHING]', 3),
 ('I kissed you.', 3),
 ('Behave exquisitely.', 2),
 ('What am I going to do with a fortune?', 2),
 ('Now and always sweet.', 2),
 ("Not like that you're not.", 2),
 ("That was a rouser, wasn't it?", 2),
 ("Actually, now that you mention it, no.  I don't.", 2),
 ('You felt something that far back?', 2),
 ('Thank you.', 2),
 ('It burned my comb too and all the towels in the bathroom.', 2),
 ("I guess we don't need glasses.", 2),
 ("It's not champagne.", 2),
 ('You started to write me.', 2),
 ('Look what we got here.', 2),
 ("Shh.  If we're very quiet the fish might come.", 2),
 ("Why didn't you let me know?", 2),
 ('oh.', 2),
 ("You're the only one I know who loves his parents.", 1),
 ('Oh, dear go

In [37]:
pd.DataFrame(get_pattern(list(sentences[0:3])))

,0,1,10,11,12,2,3,4,5,6,7,8,9
0,.+ me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,.+ .+ a,.+ .+ a,a .+,there .+,there a .+,.+ there,.+ a .+,.+ a .+,.+ a,.+ problem,is there .+,is .+,is .+ a


In [40]:
get_pattern(list(sentences[3:6]))

[{u'0': u'let .+',
  u'1': u'let me .+',
  u'10': u'.+ problem',
  u'11': u'.+ .+ me',
  u'12': u'.+ .+ me',
  u'13': u'.+ let',
  u'14': u'.+ let me',
  u'15': u'.+ me .+',
  u'16': u'.+ me .+',
  u'17': u'.+ me',
  u'18': u'.+ change',
  u'19': u'.+ it',
  u'2': u'me .+ it',
  u'20': u'well .+',
  u'21': u'the .+',
  u'22': u'the .+ .+',
  u'23': u'the .+ .+',
  u'24': u'problem .+',
  u'3': u'me .+ .+',
  u'4': u'me .+ .+',
  u'5': u'me .+',
  u'6': u'change .+',
  u'7': u'.+ the .+',
  u'8': u'.+ the .+',
  u'9': u'.+ the'},
 {u'0': u'id .+',
  u'1': u'.+ getting',
  u'10': u'.+ is',
  u'11': u'.+ is .+',
  u'12': u'.+ is .+',
  u'13': u'.+ why',
  u'14': u'.+ .+ my',
  u'15': u'.+ .+ my',
  u'16': u'.+ here',
  u'17': u'.+ my',
  u'18': u'.+ my .+',
  u'19': u'.+ my .+',
  u'2': u'.+ an',
  u'20': u'.+ was .+',
  u'21': u'.+ was .+',
  u'22': u'.+ was',
  u'23': u'what .+',
  u'24': u'is .+',
  u'25': u'an .+',
  u'26': u'why .+',
  u'27': u'here .+',
  u'28': u'here .+ .+',
  u'2

In [ ]:
set(get_pattern(list(sentences[3]))[0].values())